# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [1]:
#Check if on Linux, macOS, or Windows
def identify_platform(): 
    import platform
    info = platform.system()
    print(info)
    if info == 'Windows':
        return "Windows"
    elif info == 'Linux':
        return "Linux"
    elif info == 'macOS':
        return "macOS"
    else:#assume linux syntax
        return "Linux"
os_name = identify_platform()
# Get the datasets
#Use wget on linux

#Use curl on Microsoft Windows.
if os_name == "Windows":
    !curl.exe --output train.dat http://huang.eng.unt.edu/CSCE-5218/train.dat
    !curl.exe --output test.dat http://huang.eng.unt.edu/CSCE-5218/test.dat

#if macOS
elif os_name == "macOS":
    !curl. --output train.dat http://huang.eng.unt.edu/CSCE-5218/train.dat
    !curl --output test.dat http://huang.eng.unt.edu/CSCE-5218/test.dat
    
#if not the avove, assume linux tools are available
else:
    !wget http://huang.eng.unt.edu/CSCE-5218/test.dat
    !wget http://huang.eng.unt.edu/CSCE-5218/train.dat




Windows


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 11244  100 11244    0     0   208k      0 --:--:-- --:--:-- --:--:--  211k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2844  100  2844    0     0  32050      0 --:--:-- --:--:-- --:--:-- 32318


In [2]:
# Take a peek at the datasets

#On linux
if os_name == "Linux" or os_name == "macOS":
    !head train.dat
    !head test.dat
    
#Windows has no "head" command, so print 10 lines of data for each instead. 
elif os_name == "Windows":
    def head(f, lines = 10):
        with open(f, mode="r") as file:
            file = file.read()
            file = file.split("\n")[0:lines]
            for line in file:
                print(line)
            print("\n")
            
    head("train.dat")
    head("test.dat")
    

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0


A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0




### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this. 

In [3]:
import math
import itertools
import re

# Corpus reader, all columns but the last one are coordinates;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        ###################################Bug fix########################################
        #original tutorial creates a type error in the following lines by concating an integer to a list of lists. 
        #this code is altered from the tutorial to avoid this bug and concat the bias into the intended list
        #remove brackets that got in the way of concatination and append to a list rather than list of lists
        instance = list(map(int, instance.strip().split('\t')))
        
        # Add a dummy input so that w0 becomes the bias, then create the intended list
        instance = [[-1] + instance]
        ############################################################################
        data += instance
    #check datatype (for debugging)
    #print(f"Datatype: {type(data)}, count = {len(data)}")
    return data

#check datatype. Since we are not using numpy, the datatype is technically now a list represenitng a 1d array
read_data("test.dat")

def dot_product(array1, array2):
    #Return dot product of array 1 and array 2. 
    #Since these are lists, that takes care of flattening the arrays. Sum and zip are in-built python functions, so I am assuming they are allowed.
    #Use list comprehension to multiply matching elements, then take the sum of the resulting list. This manuel method is NOT performant and could be made much more efficient.
    product = sum([arr1*arr2 for (arr1,arr2) in zip(array1,array2)])
    return product 

print(f"Verifying dot prod method: [0,1,2,3],[0,1,2,3] = {0*0 + 1*1 + 2*2 + 3*3}. My dot_product method = ",dot_product([0,1,2,3],[0,1,2,3]))

def sigmoid(x):
    #Return outpout of sigmoid function on x. 
    sigmoid = (1)/(1 + math.e**(-x))
    return sigmoid

# The output of the model, which for the perceptron is 
# the sigmoid function applied to the dot product of 
# the instance and the weights
def output(weight, instance):
    #return the output of the model 
    model_output = sigmoid(dot_product(instance,weight))
    return model_output

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    #return the prediction of the model
    if output(weights,instance) >= 0.5: return 1
    else: return 0


# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    # You do not to write code like this, but get used to it
    correct = sum([1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances])
    return correct * 100 / len(instances)


# Train a perceptron with instances and hyperparameters:
#       lr (learning rate) 
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):
    #name this step: 
    #This step initializes the weights to each of the inputs and assigns them a value of zero. Note: this function threw an error in the original code because the bias did not get integrated into the intended list but was rather seperate in the dataset as a standalone integer.
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            #name these steps
            #Multiply the weights by the inputs
            in_value = dot_product(weights, instance)
            #Apply the sigmoid to the product of weights*inputs to get output
            output = sigmoid(in_value)
            #Caclulate the diference betwen the true values and the predicted output from the above sigmoid function.
            error = instance[-1] - output
            #name these steps
            #For every weight, update the value to minimize the loss. LR slows down the step size/learning rate, error, instance, and output were calculated above.
            for i in range(0, len(weights)):
                weights[i] += lr * error * output * (1-output) * instance[i]
    #return the set of all weights to repeat the process and improve.
    return weights

Verifying dot prod method: [0,1,2,3],[0,1,2,3] = 14. My dot_product method =  14


## Run it

In [4]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)

Answer: predict() returns either a 1 or a 0. While this binary output is eventually our goal for this perceptron, sigmid is a far better function for calculating loss. If we can only calculate loss based on 1 and 0 and nothing in-between, then we will not be able to get as good training convergence because we will be losing information. However, we do end up using predict() to calculate our accuracy metrics at the end (unrelated to training).


### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above) 

In [5]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]     # number of epochs
lr_array = [0.005, 0.01, 0.05]        # learning rate

history = []

for lr in lr_array:
  for tr_size in tr_percent:
    for epochs in num_epochs:
      for _ in range(epochs):
          size =  round(len(instances_tr)*tr_size/100)
          pre_instances = instances_tr[0:size]
          weights = train_perceptron(pre_instances, lr, epochs)
          accuracy = get_accuracy(weights, instances_te)
      ##collect data for fine-tuning
      history.append([lr,epochs,len(pre_instances),accuracy])
      print(f"#tr: {len(pre_instances):0}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
              f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 20, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 40, epochs: 100, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
#tr: 100, epochs:  20, lea

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)



Instead of graphing the results, we might get better multi-variate comparison by simply ordering the list by accuracy from low to high.  

In [6]:
#Convert our training history to pandas for efficient sorting.
try:
    import pandas as pd
except:
    !pip install pandas #install and import pandas, if it is not already installed.
    import pandas as pd

#sort data by accuracy, since that is what is most relevant for training accuracy/convergence.
history_dataframe = pd.DataFrame(history)
history_dataframe.rename(columns={0: 'lr', 1:'epochs', 2:'samples',3:'accuracy'}, inplace=True)
history_dataframe.sort_values('accuracy',inplace=True)

In [7]:
#check the lowest 10 performing hyperperameter combinations
history_dataframe.head(10)

,lr,epochs,samples,accuracy
64,0.050,100,20,64.0
71,0.050,10,100,67.0
18,0.005,50,200,67.0
61,0.050,10,20,68.0
32,0.010,20,20,68.0
33,0.010,50,20,68.0
34,0.010,100,20,68.0
35,0.010,5,40,68.0
36,0.010,10,40,68.0
37,0.010,20,40,68.0


In [8]:
#check the 10 highest performing hyperperameter combinations
history_dataframe.tail(10)

,lr,epochs,samples,accuracy
81,0.05,10,300,78.0
83,0.05,50,300,78.0
49,0.01,100,200,78.0
53,0.01,50,300,78.0
82,0.05,20,300,79.0
59,0.01,100,400,80.0
88,0.05,50,400,80.0
54,0.01,100,300,80.0
87,0.05,20,400,80.0
89,0.05,100,400,80.0


A) Starting at the highest performers, we can answer the first question: training with all the data is not necessary to achieve the highest accuracy of 80%. However, this comes at a big cost in terms of required epochs. Training at 20 epochs and 400 samples (20*400 = 8000 steps) was less comptationally expensive than 300 samples at 100 epochs (300*100 = 30000 steps)

B) While the first one does use less training data, it also has a much higher learning rate (x10). The weights are allowed to change much faster than the second example, and this appears to be sufficient to outperfomr the first. By comparison, we can prove that the second example was undertrained and might benifit from more epochs than the first. 


C) Based on the tested samples, we never get beyond 80%. Empirically, we have no reason to believe we can fit a better linear line though the data. The data is simply too complicated for one neuron to master. Generally, lines of best fit should not take too long to converge compared to non-linear models. Bellow, there is a proof that increasing epochs will not help us get better performance. Since we are already using 100% of data in the top performers and the lowest learning rate did not improve the performance, this is further confirmed.

D) Looking at both highest and lowest performers, it becomes apparant that epochs only improve performance up to a certain point at which adding more becomes either irrelevent or counterproductive. As an example, two of the top performers were completely identical except for the epochs of 20 and 50. Yet both got the apparant maximum score of 80. Since this perceptron is so simple, it is unlikely to overfit. However, the linear nature of the model can only fit straight lines, which means that continuing to refit after reaching the global peak is a waste of time. This concept can be found thoughout the entire dataset. Even though more epochs can improve the model performance, the benifits appear capped by samples. Lower learning rates, on the other hand, do directly require higher epochs.